## Import Libraries

In [9]:
#%load_ext cudf.pandas

The cudf.pandas extension is already loaded. To reload it, use:
  %reload_ext cudf.pandas


In [10]:
#import libraries
import pandas as pd
import os

## Created Functions

In [11]:
#functions used later

#Store cleaned or semi construscted dataframes to a csv
def store_file(file_name,version): #parameters to input 
    water_system_dict[file_name].to_csv(f"../Clean_Data/{file_name}_V{version}.csv")
    return print("file has been saved")

#Display missing column and their details, where you just need to input the file name for results
def missing_column_display(check_file):
    columns_with_missing = water_system_dict[check_file].columns[water_system_dict[check_file].isnull().sum() > 1] # find all data points with more than 1 missing value
    display_missing_columns=water_system_dict[check_file][columns_with_missing] #variable to show only null columns
    return display_missing_columns

#Correct column size
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df


## Extract Data

In [12]:
#obtain path to folder related to water system data
path = "../Unclean_Data"
dir_list = os.listdir(path) # collect all file names within folder

In [13]:
#initialize dictionary to hold all files related water system data
water_system_dict={}
for file in dir_list:
     if file.endswith(".csv"): # remove csv extension for naming purposes
          file_name=file.replace(".csv","")
     water_system_dict[file_name]= pd.read_csv(f"../Unclean_Data/{file}", encoding='ISO-8859-1') #hold each file in the dictionary

In [14]:
#check for null values in the dataset
for null_check in water_system_dict:
    print("null count for" , null_check, "is:", water_system_dict[null_check].isnull().sum().sum())

null count for Basic_Facility_Report_2023 is: 42908
null count for Chemical_Report_2023 is: 705830
null count for Flow_Data_Report_2023 is: 5476
null count for Microbiological_Report_2023 is: 375222
null count for Plant_Treatment_Report_2023 is: 3853


### Each file looks like it has numerous null values that will need to be individually analyzed to determine how to handle them, whether that be dropping columns, rows, or filling in

## Microbiological Cleaning

In [15]:
water_system_dict["Microbiological_Report_2023"].isnull().sum()

PWSID                   0
OFFICE                  0
SYSTEMNAME              0
SYSTEMTYPE              0
SYSTEMSTATUS            0
LABID                   0
SAMPLETYPE              0
RESULTS             87044
COLIFORMPRESENT     87207
SAMPLENUMBER        83626
LINKTO              86078
SAMPLESREQUIRED         0
SAMPLESTAKEN            0
SAMPLEDATE              0
METHOD                  0
CHLORINERESIDUAL    31267
dtype: int64

In [16]:
water_system_dict["Microbiological_Report_2023"].shape

(90644, 16)

#### The columns results, coliformpresent,samplenumber,and linkto seem to be mostly null indicating that it might be tolerable to drop them completely but the documentation should be checked to see if it can be filled instead

In [17]:
missing_column_display("Microbiological_Report_2023")

,RESULTS,COLIFORMPRESENT,SAMPLENUMBER,LINKTO,CHLORINERESIDUAL
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,0.99
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
90639,NaN,NaN,NaN,NaN,NaN
90640,NaN,NaN,NaN,NaN,NaN
90641,NaN,NaN,NaN,NaN,NaN
90642,NaN,NaN,NaN,NaN,NaN


In [18]:
water_system_dict["Microbiological_Report_2023"]["RESULTS"].value_counts()

RESULTS
NaN      87044
1.0       2475
0.0       1087
999.0       38
Name: count, dtype: int64

### The output values are hard to understand as they are 0,1, and 999 which I originally presumed to be a binary feature but seems like it isn't; this column relates to the results of chemical analysis,stated by the pws_data_dictionary, the column may be relevant, but I don't want to utilize something that might not be properly functioning

In [19]:
water_system_dict["Microbiological_Report_2023"]["COLIFORMPRESENT"].value_counts()

COLIFORMPRESENT
0    2463
N     869
E     104
F       1
Name: count, dtype: int64

### Based on the pws_data_dictionary we can assume that nan values can be replaced by the string "N" which implies that no analysis was performed

In [20]:
water_system_dict["Microbiological_Report_2023"]["COLIFORMPRESENT"].fillna("N",inplace=True)

### The samplenumber seems to be a secondary key, that isn't relevant to the overall structure of our data, and can be dropped

In [21]:
water_system_dict["Microbiological_Report_2023"]["LINKTO"].value_counts().head(10)

LINKTO
001       48
002       42
007       29
004       19
006       18
231028    16
A1        16
008       15
853-01    15
9002      15
Name: count, dtype: int64

### The linkto column most likely has relation to an outside dataset as the pws_data_dictionary states it associates follow up samples to positive samples, so it might be a foreign key or some sort of class-labeling; will need to do more research on this one.

In [22]:
water_system_dict["Microbiological_Report_2023"]["CHLORINERESIDUAL"].value_counts().head(10)

CHLORINERESIDUAL
NaN    31267
0.0    12743
1.0     3651
0.8     2300
1.2     2096
0.6     1455
0.7     1408
2.0     1333
1.1     1328
1.5     1300
Name: count, dtype: int64

### The residual measure's results can be utilized for analysis but while it has a large amount of null values, it can be interpreted that the null values can be replaced by something like "N/A", but wouldn't respect the dtype of int64 so it might be safer to assign a numeric value to it so the column can function properly.

In [23]:
#We can fill all the values after the values with the previous values as they mostly like hold some relation with respect to how the pwsids are sorted
water_system_dict["Microbiological_Report_2023"]["CHLORINERESIDUAL"].fillna(method="ffill",inplace=True)
water_system_dict["Microbiological_Report_2023"]["CHLORINERESIDUAL"].fillna(0,inplace=True) #The initial null values will not be filled, so it would be safe to replace it with 0
water_system_dict["Microbiological_Report_2023"]["CHLORINERESIDUAL"].isnull().sum() #Verify there are no more null values

/home/haslett/miniforge3/envs/rapids-24.10/lib/python3.12/site-packages/cudf/core/frame.py:731: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  warnings.warn(


np.int64(31267)

In [24]:
store_file("Microbiological_Report_2023",1)

file has been saved


In [25]:
#find all missing columns that are still mostly null
columns_with_missing = water_system_dict["Microbiological_Report_2023"].columns[
    water_system_dict["Microbiological_Report_2023"].isnull().sum() > 1]

#drop those columns that are still prominantly null
water_system_dict["Microbiological_Report_2023"].drop(columns=columns_with_missing,inplace=True)

In [26]:
water_system_dict["Microbiological_Report_2023"]

,PWSID,OFFICE,SYSTEMNAME,SYSTEMTYPE,SYSTEMSTATUS,LABID,SAMPLETYPE,COLIFORMPRESENT,SAMPLESREQUIRED,SAMPLESTAKEN,SAMPLEDATE,METHOD
0,1030050,NWD,BAY COUNTY WATER SYSTEM,COMMUNITY,ACTIVE,E41292,D,N,30,9,12-Jan-23,CLL
1,1030050,NWD,BAY COUNTY WATER SYSTEM,COMMUNITY,ACTIVE,E41292,D,N,30,9,19-Jan-23,CLL
2,1030050,NWD,BAY COUNTY WATER SYSTEM,COMMUNITY,ACTIVE,E41292,D,N,30,9,24-Jan-23,CLL
3,1030050,NWD,BAY COUNTY WATER SYSTEM,COMMUNITY,ACTIVE,E41292,D,N,30,9,2-Feb-23,CLL
4,1030050,NWD,BAY COUNTY WATER SYSTEM,COMMUNITY,ACTIVE,E41292,D,N,30,9,9-Feb-23,CLL
...,...,...,...,...,...,...,...,...,...,...,...,...
90639,6605038,CD,CSU WTP NO. 1,COMMUNITY,ACTIVE,E83141,R,N,20,2,9-Aug-23,MF
90640,6605038,CD,CSU WTP NO. 1,COMMUNITY,ACTIVE,E83141,R,N,20,2,13-Sep-23,MF
90641,6605038,CD,CSU WTP NO. 1,COMMUNITY,ACTIVE,E83141,R,N,20,2,1-Oct-23,MF
90642,6605038,CD,CSU WTP NO. 1,COMMUNITY,ACTIVE,E83141,R,N,20,2,13-Nov-23,MF


## Chemical Cleaning

In [27]:
water_system_dict["Chemical_Report_2023"].isnull().sum()

PWSID                    0
OFFICE                   0
MAILINGNAME              0
SYSTEMTYPE               0
POPULATION               0
SYSTEMSTATUS             0
SUBPARTH                 0
DESIGNCAP                0
CONSECIND                0
INSPINIT              3077
CONTAMGROUP              0
CONTAMCODE               0
CONTAMDESC               0
SAMPLETYPE               0
RESULTS                  0
UOM                   3641
MCL                  15080
SAMPLEDATE               0
SAMPTIMESTAMP            0
USERNAME                 0
LOCATIONCODE             0
ENTRYPOINT           28489
LAB                      0
METHOD                   0
ANALYSISDATE          6497
MDL                  84941
RDL                  83311
SAMPLENUMBER        146659
CHLORINERESIDUAL    145677
MAXQTR              154779
REMARKS              33679
dtype: int64

In [28]:
missing_column_display("Chemical_Report_2023")

,INSPINIT,UOM,MCL,ENTRYPOINT,ANALYSISDATE,MDL,RDL,SAMPLENUMBER,CHLORINERESIDUAL,MAXQTR,REMARKS
0,LJ,UG/L,60.4999,NaN,8-Oct-23,NaN,NaN,NaN,NaN,3.0,"W17 - 17TH @ COVE, NW SIDE OF INTERSECT"
1,LJ,UG/L,60.4999,NaN,9-Nov-23,NaN,NaN,NaN,NaN,3.0,"W17 - 17TH @ COVE, NW SIDE OF INTERSECT"
2,LJ,UG/L,60.4999,NaN,9-Mar-23,0.9,NaN,NaN,NaN,3.0,W17
3,LJ,UG/L,60.4999,NaN,14-Apr-23,NaN,NaN,NaN,NaN,3.0,"W17 - 17TH @ COVE, NW SIDE OF INTERSECT"
4,LJ,UG/L,60.4999,NaN,8-Oct-23,NaN,NaN,NaN,NaN,3.0,RW21 - 2521 PALO ALTO AVE
...,...,...,...,...,...,...,...,...,...,...,...
159836,NaN,UG/L,149.9999,Y,3-Mar-23,NaN,0.5499,NaN,NaN,NaN,POE
159837,NaN,UG/L,1.4999,Y,3-Mar-23,NaN,0.5499,NaN,NaN,NaN,POE
159838,NaN,UG/L,1499.9999,Y,3-Mar-23,NaN,0.5499,NaN,NaN,NaN,POE
159839,NaN,UG/L,749.9999,Y,3-Mar-23,NaN,0.5499,NaN,NaN,NaN,POE


In [29]:
water_system_dict["Chemical_Report_2023"].shape

(159841, 31)

In [30]:
columns_with_missing = water_system_dict["Chemical_Report_2023"].columns[
    water_system_dict["Chemical_Report_2023"].isnull().sum() > 1]

In [31]:
water_system_dict["Chemical_Report_2023"][columns_with_missing]

,INSPINIT,UOM,MCL,ENTRYPOINT,ANALYSISDATE,MDL,RDL,SAMPLENUMBER,CHLORINERESIDUAL,MAXQTR,REMARKS
0,LJ,UG/L,60.4999,NaN,8-Oct-23,NaN,NaN,NaN,NaN,3.0,"W17 - 17TH @ COVE, NW SIDE OF INTERSECT"
1,LJ,UG/L,60.4999,NaN,9-Nov-23,NaN,NaN,NaN,NaN,3.0,"W17 - 17TH @ COVE, NW SIDE OF INTERSECT"
2,LJ,UG/L,60.4999,NaN,9-Mar-23,0.9,NaN,NaN,NaN,3.0,W17
3,LJ,UG/L,60.4999,NaN,14-Apr-23,NaN,NaN,NaN,NaN,3.0,"W17 - 17TH @ COVE, NW SIDE OF INTERSECT"
4,LJ,UG/L,60.4999,NaN,8-Oct-23,NaN,NaN,NaN,NaN,3.0,RW21 - 2521 PALO ALTO AVE
...,...,...,...,...,...,...,...,...,...,...,...
159836,NaN,UG/L,149.9999,Y,3-Mar-23,NaN,0.5499,NaN,NaN,NaN,POE
159837,NaN,UG/L,1.4999,Y,3-Mar-23,NaN,0.5499,NaN,NaN,NaN,POE
159838,NaN,UG/L,1499.9999,Y,3-Mar-23,NaN,0.5499,NaN,NaN,NaN,POE
159839,NaN,UG/L,749.9999,Y,3-Mar-23,NaN,0.5499,NaN,NaN,NaN,POE


In [32]:
water_system_dict["Chemical_Report_2023"]["INSPINIT"].value_counts().head(10)

INSPINIT
RFP    9572
JL     9153
MN     6573
MFC    6390
DS     4833
PA     4568
PLS    4216
DG     4124
JMF    3990
SBC    3769
Name: count, dtype: int64

### The INSPINIT columns is just inspector initials according to pws_data_dictionary, that mostly likey won't be necessary, but the null values can simply be replaced with "NA" 

In [33]:
water_system_dict["Chemical_Report_2023"]["INSPINIT"].fillna("NA",inplace=True)

In [34]:
water_system_dict["Chemical_Report_2023"]["UOM"].value_counts()

UOM
UG/L           93551
MG/L           46990
NTU             6497
LN(10)          5108
PCI/L           2743
COLOR UNITS     1053
NG/L             200
MF/L              58
Name: count, dtype: int64

### According to pws_data_dictionary, UOM is the unit of measurement associated with the sample results, and the nall values for this can be replaced with NA as well

In [35]:
water_system_dict["Chemical_Report_2023"]["UOM"].fillna("NA",inplace=True)

In [36]:
water_system_dict["Chemical_Report_2023"]["MCL"].value_counts().head(10)

MCL
1.4999      19750
NaN         15080
3.4999      10644
0.2499       7939
10.4999      6683
2.4999       6194
80.4999      5897
60.4999      5838
149.9999     5823
5.4999       5374
Name: count, dtype: int64

In [37]:
water_system_dict["Chemical_Report_2023"]["MCL"]

0           60.4999
1           60.4999
2           60.4999
3           60.4999
4           60.4999
            ...    
159836     149.9999
159837       1.4999
159838    1499.9999
159839     749.9999
159840     149.9999
Name: MCL, Length: 159841, dtype: float64

### Based on the pws_data_dictionary, MCL is the maximum contaminent level for compliance check, which can be replaced by the mean as it would handle the vast range of values well.

In [38]:
water_system_dict["Chemical_Report_2023"]["MCL"].fillna(value=water_system_dict["Chemical_Report_2023"]["MCL"].mean(),inplace=True)

In [39]:
water_system_dict["Chemical_Report_2023"]["ENTRYPOINT"].value_counts()

ENTRYPOINT
Y    128021
N      3331
Name: count, dtype: int64

### It can be assumed the samples taken at entrypoint, as defined by the pws_data_dictionary, that are null values can be replaced by N, as it would seem to important to not note it as Y

In [40]:
water_system_dict["Chemical_Report_2023"]["ENTRYPOINT"].fillna("N",inplace=True)

In [41]:
water_system_dict["Chemical_Report_2023"]["ANALYSISDATE"]

0          8-Oct-23
1          9-Nov-23
2          9-Mar-23
3         14-Apr-23
4          8-Oct-23
            ...    
159836     3-Mar-23
159837     3-Mar-23
159838     3-Mar-23
159839     3-Mar-23
159840     3-Mar-23
Name: ANALYSISDATE, Length: 159841, dtype: object

### We can apply a ffill as the index locations seem to be adjacent to one another, and would be most aligned to nan values for date of analysis

In [42]:
water_system_dict["Chemical_Report_2023"]["ANALYSISDATE"].fillna(method="ffill",inplace=True)

/home/haslett/miniforge3/envs/rapids-24.10/lib/python3.12/site-packages/cudf/pandas/fast_slow_proxy.py:26: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return fn(*args, **kwargs)


In [43]:
water_system_dict["Chemical_Report_2023"]["MDL"].isnull().sum()

np.int64(84941)

In [44]:
water_system_dict["Chemical_Report_2023"]["MDL"].value_counts().head(10)

MDL
NaN     84941
0.50     4037
0.20     3509
0.10     2659
0.26     1554
0.00     1396
1.00     1325
0.28     1204
0.09     1200
0.44     1166
Name: count, dtype: int64

### The mdl is the method detection limit as stated by the pws_data_dictionary, which leads me to believe that the mean would be effective here as well

In [45]:
water_system_dict["Chemical_Report_2023"]["MDL"].fillna(value=water_system_dict["Chemical_Report_2023"]["MDL"].mean(),inplace=True)

In [46]:
water_system_dict["Chemical_Report_2023"]["RDL"].isnull().sum()

np.int64(83311)

In [47]:
water_system_dict["Chemical_Report_2023"]["RDL"].value_counts()

RDL
NaN       83311
0.5499    31011
0.1499    10355
0.0249     5946
0.2499     5903
1.4999     4674
0.6499     3099
0.0149     2995
0.0449     2963
0.0749     1481
0.4499     1481
2.4999     1467
0.9499     1464
9.4999     1463
6.4999     1460
0.6670      401
0.0055      200
3.4999      167
Name: count, dtype: int64

### The rdl, known as the Regulatory detection limit according to pws_data_dictionary, would appear to be similar to mdl in use case, so using the mean here should be fine as well

In [48]:
water_system_dict["Chemical_Report_2023"]["RDL"].fillna(value=water_system_dict["Chemical_Report_2023"]["RDL"].mean(),inplace=True)

In [49]:
water_system_dict["Chemical_Report_2023"]["SAMPLENUMBER"]

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
159836    NaN
159837    NaN
159838    NaN
159839    NaN
159840    NaN
Name: SAMPLENUMBER, Length: 159841, dtype: object

### Since a majority of the sample numbers appear to be missing, it would be best to drop the column as a whole even though it would serve well as a foreign key

In [50]:
water_system_dict["Chemical_Report_2023"].drop(columns="SAMPLENUMBER",inplace=True)

In [51]:
water_system_dict["Chemical_Report_2023"]["CHLORINERESIDUAL"].value_counts().head(10)

CHLORINERESIDUAL
NaN    145677
1.0       609
2.0       482
1.2       455
0.8       420
0.5       359
1.5       344
0.6       324
0.9       315
2.8       301
Name: count, dtype: int64

In [52]:
(water_system_dict["Chemical_Report_2023"]["CHLORINERESIDUAL"].isnull().sum())/len(water_system_dict["Chemical_Report_2023"]["CHLORINERESIDUAL"])

np.float64(0.9113869407723926)

In [53]:
null_count = water_system_dict["Chemical_Report_2023"]["CHLORINERESIDUAL"].isnull().sum()
total_count = len(water_system_dict["Chemical_Report_2023"]["CHLORINERESIDUAL"])
null_proportion = null_count / total_count

print(f"Proportion of null values in CHLORINERESIDUAL: {null_proportion:.2%}")

Proportion of null values in CHLORINERESIDUAL: 91.14%


### According to pws_data_dictionary the chlorine residual is the residual measured for the sample, and is actually an important feature for analysis, but there is a large amount of data missing, ~91%, it might be safer to replace it with a binary column of whether or not there is data present

In [54]:
#binary column developed that renders anything greater than 0 in the original column to 1, and 0 and null values to 0
water_system_dict["Chemical_Report_2023"]["CHLORINERESIDUAL_BINARY"] = water_system_dict["Chemical_Report_2023"]["CHLORINERESIDUAL"].gt(0).astype(int)

In [55]:
water_system_dict["Chemical_Report_2023"]["MAXQTR"].value_counts()

MAXQTR
NaN    154779
3.0      3089
2.0       721
4.0       660
1.0       592
Name: count, dtype: int64

### The max qtr is the quarter the sample has to be taken according to pws_data_dicitonary, which doesn't seem to be a relevant feature, when the sampledate can be utilized, which also had fewer na values in it originally

In [56]:
water_system_dict["Chemical_Report_2023"].drop(columns="MAXQTR",inplace=True)

In [57]:
water_system_dict["Chemical_Report_2023"]["REMARKS"]

0         W17 - 17TH @ COVE, NW SIDE OF INTERSECT
1         W17 - 17TH @ COVE, NW SIDE OF INTERSECT
2                                             W17
3         W17 - 17TH @ COVE, NW SIDE OF INTERSECT
4                       RW21 - 2521 PALO ALTO AVE
                           ...                   
159836                                        POE
159837                                        POE
159838                                        POE
159839                                        POE
159840                                        POE
Name: REMARKS, Length: 159841, dtype: object

### The remarks are just comments the inspectors left according to pws_data_dictionary, and could be useful, whereas the nan values can be replaced with "no comment"

In [58]:
water_system_dict["Chemical_Report_2023"]["REMARKS"].fillna("No Comment",inplace=True)

In [59]:
columns_with_missing = water_system_dict["Chemical_Report_2023"].columns[
    water_system_dict["Chemical_Report_2023"].isnull().sum() > 1]

In [60]:
water_system_dict["Chemical_Report_2023"][columns_with_missing]

,MCL,MDL,RDL,CHLORINERESIDUAL
0,60.4999,NaN,NaN,NaN
1,60.4999,NaN,NaN,NaN
2,60.4999,0.9,NaN,NaN
3,60.4999,NaN,NaN,NaN
4,60.4999,NaN,NaN,NaN
...,...,...,...,...
159836,149.9999,NaN,0.5499,NaN
159837,1.4999,NaN,0.5499,NaN
159838,1499.9999,NaN,0.5499,NaN
159839,749.9999,NaN,0.5499,NaN


In [61]:
store_file("Chemical_Report_2023",1)

file has been saved


## Flow Data Cleaning

In [62]:
water_system_dict["Flow_Data_Report_2023"].isnull().sum()

PWSID               0
OFFICE              0
SYSTEMNAME          0
DESIGNCAPCITY    4638
PLANTNUMBER         0
PLANTSTATUS         0
REPORTDATE          0
MAX_TREATED       419
AVG_TREATED       419
dtype: int64

In [63]:
missing_column_display("Flow_Data_Report_2023").head(50)

,DESIGNCAPCITY,MAX_TREATED,AVG_TREATED
0,60000000.0,2770000.0,1894000.0
1,60000000.0,2288000.0,2074110.0
2,60000000.0,25870000.0,21240000.0
3,60000000.0,28060000.0,24257700.0
4,60000000.0,30970000.0,23720000.0
5,60000000.0,30580000.0,27155700.0
6,60000000.0,31220000.0,23890000.0
7,60000000.0,36590000.0,29210600.0
8,60000000.0,27600000.0,22520000.0
9,60000000.0,27320000.0,22180000.0


In [64]:
water_system_dict["Flow_Data_Report_2023"]["DESIGNCAPCITY"].value_counts().head(10)

DESIGNCAPCITY
NaN        4638
1.0        1598
5000.0     1165
36000.0     935
10000.0     911
10800.0     664
18000.0     655
28800.0     629
72000.0     617
49000.0     583
Name: count, dtype: int64

### A ffill would work here as design capacity is just the design capacity of the plant according to pws_data_dictionary, but mostly the appearance that data points are similar to their neighbors

In [65]:
water_system_dict["Flow_Data_Report_2023"]["DESIGNCAPCITY"].fillna(method="ffill",inplace=True)

/home/haslett/miniforge3/envs/rapids-24.10/lib/python3.12/site-packages/cudf/core/frame.py:731: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  warnings.warn(


In [66]:
water_system_dict["Flow_Data_Report_2023"]["MAX_TREATED"].value_counts()

MAX_TREATED
0.000000e+00    1687
NaN              419
1.000000e+03     384
4.000000e+02     323
1.000000e+00     319
                ... 
4.254915e+08       1
1.123259e+09       1
1.596000e+09       1
1.667000e+09       1
1.917000e+09       1
Name: count, Length: 13584, dtype: int64

### According to pws_data_dictionary, the max treated is the highest daily average gallons of water per day in an one month period, in which the nan values can be replaced by 0, since this is an important feature to note, so why wouldn't it be recorded?. It is apparent that 0 is also the mode within the feature as well.

In [67]:
water_system_dict["Flow_Data_Report_2023"]["MAX_TREATED"].fillna(0,inplace=True)

In [68]:
water_system_dict["Flow_Data_Report_2023"]["AVG_TREATED"].value_counts().head(10)

AVG_TREATED
0.0       1707
NaN        419
1.0        342
1000.0      64
400.0       59
32.0        58
2000.0      55
3000.0      52
100.0       44
300.0       43
Name: count, dtype: int64

### According to pws_data_dictionary, the average treated is the average number of gallons per day of water processed by a plant during a month, which can follow the same rhetoric as the max treated, where the nan values can be replaced by 0, as it's present as the mode as well.

In [69]:
water_system_dict["Flow_Data_Report_2023"]["AVG_TREATED"].fillna(0,inplace=True)

In [70]:
store_file("Flow_Data_Report_2023",1)

file has been saved


## Basic Facility Cleaning

In [71]:
water_system_dict["Basic_Facility_Report_2023"].isnull().sum()

DISTRICT                 0
COUNTY                   0
OFFICE                   0
EMAIL                 3223
PWSID                    0
SYSTEMTYPE               0
CATEGORYCLASS            0
SURFACESOURCE            0
GROUNDSOURCE             0
MAILINGNAME              0
ADDRESS1                 0
ADDRESS2              5689
CITY                     0
SYSTEMSTATE              0
ZIP                      0
ZIPFOUR               5659
PHONE                  462
PHONEEXT              5841
CONTACTNAME           1473
OWNERNAME              235
OWNERADDRESS1          273
OWNERADDRESS2         5344
OWNERCITY              272
OWNERSTATE             272
OWNERZIP               273
OWNERZIP4             5453
OWNERPHONE             558
OWNERPHONEEXT         5710
OWNERTYPE                0
POPULATIONSERVED         0
DESIGNCAPACITY           0
SERVICECONNECTIONS       0
LASTINSPECTION        1075
LASTSANITARYSURVEY    1096
FEEGROUPCODE             0
ANNUALFEE                0
dtype: int64

In [72]:
missing_column_display("Basic_Facility_Report_2023")

,EMAIL,ADDRESS2,ZIPFOUR,PHONE,PHONEEXT,CONTACTNAME,OWNERNAME,OWNERADDRESS1,OWNERADDRESS2,OWNERCITY,OWNERSTATE,OWNERZIP,OWNERZIP4,OWNERPHONE,OWNERPHONEEXT,LASTINSPECTION,LASTSANITARYSURVEY
0,slathrop@baycountyfl.gov,NaN,NaN,8.502485e+09,NaN,BOBBY GIBBS,BEN BLITCH,3410 TRANSMITTER ROAD,NaN,PANAMA CITY,FL,32404.0,NaN,8.502485e+09,NaN,27-Dec-13,9-Dec-21
1,WFRYE@CITYOFCALLAWAY.COM;,NaN,9542.0,8.508711e+09,NaN,NaN,EDDIE COOK,6601 E. HWY 22,NaN,CALLAWAY,FL,32404.0,9542.0,8.502157e+09,NaN,12-Nov-13,12-Aug-21
2,jbaldwin@springfield.fl.gov;mgrice@springfield...,NaN,NaN,8.508728e+09,NaN,MILES GRICE,RALPH HAMMOND,408 SCHOOL AVE.,NaN,SPRINGFIELD,FL,32401.0,NaN,8.508728e+09,110.0,3-Jan-20,9-Dec-21
3,gkidwell@cityoflynnhaven.com,NaN,NaN,8.502650e+09,NaN,CHRIS LIGHTFOOT,GREG KIDWELL,825 OHIO AVE.,NaN,LYNN HAVEN,FL,32444.0,NaN,8.502652e+09,NaN,7-Jun-21,22-Nov-21
4,g.davis@mexicobeachgov.com;,ATTN.: GLENN DAVIS,NaN,8.506483e+09,NaN,RICHARD (GLENN) DAVIS,CHRIS HUBBARD,PO BOX 13425,NaN,MEXICO BEACH,FL,32410.0,NaN,8.506486e+09,3.0,18-Mar-24,6-Feb-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5911,NaN,NaN,NaN,3.525690e+09,NaN,NaN,DR. JOSEPH NOVAK,PO BOX 295,NaN,BUSHNELL,FL,33513.0,NaN,3.525690e+09,NaN,NaN,NaN
5912,NaN,NaN,NaN,3.527938e+09,NaN,NaN,KIRK LEOPARD,6502 SW 102ND AVE.,NaN,BUSHNELL,FL,33513.0,NaN,3.527938e+09,NaN,NaN,NaN
5913,NaN,NaN,NaN,3.527938e+09,NaN,NaN,BILLY MURPHY,6502 SW 102ND AVE.,NaN,BUSHNELL,FL,33513.0,NaN,3.527938e+09,NaN,NaN,NaN
5914,NaN,NaN,NaN,3.526894e+09,NaN,NaN,BRUCE ATKINSON,319 E. ANDERSON AVE.,NaN,BUSHNELL,FL,33513.0,NaN,3.526894e+09,6836.0,NaN,NaN


In [73]:
#If an email isn't present you can just state that there was none
water_system_dict["Basic_Facility_Report_2023"]["EMAIL"].fillna("No Email",inplace=True)

In [74]:
#The address 2 line isn't that important as address1, and it can easily be replaced by "no address"
water_system_dict["Basic_Facility_Report_2023"]["ADDRESS2"]

0                       NaN
1                       NaN
2                       NaN
3                       NaN
4       ATTN.:  GLENN DAVIS
               ...         
5911                    NaN
5912                    NaN
5913                    NaN
5914                    NaN
5915                    NaN
Name: ADDRESS2, Length: 5916, dtype: object

In [75]:
water_system_dict["Basic_Facility_Report_2023"]["ADDRESS2"].fillna("No Address",inplace=True)

In [76]:
#I have to look back at this 
water_system_dict["Basic_Facility_Report_2023"]["ZIPFOUR"].nunique()

243

In [77]:
water_system_dict["Basic_Facility_Report_2023"][["PHONE","PHONEEXT"]].value_counts().head(10)

PHONE            PHONEEXT
<NA>             <NA>        461
4.0762969e+09    <NA>         44
3.867558887e+09  <NA>         30
3.523478228e+09  <NA>         21
8.505622115e+09  <NA>         15
3.526221171e+09  <NA>         14
8.004767574e+09  <NA>         13
8.635346039e+09  <NA>         13
7.278488292e+09  <NA>         10
8.63808132e+09   <NA>         10
Name: count, dtype: int64

### The phone details don't seem to be relevant to an overall analyis and I could end up dropping them

In [78]:
#Replace null contact values with "Contact_Name_Unknown"
water_system_dict["Basic_Facility_Report_2023"]["CONTACTNAME"].fillna("Contact_Name_Unknown",inplace=True)

In [79]:
#Owner details don't seem to be pertinent to analysis, and can be replaced with the value "unknown"
water_system_dict["Basic_Facility_Report_2023"][["OWNERNAME","OWNERADDRESS1","OWNERADDRESS2","OWNERCITY","OWNERSTATE","OWNERZIP","OWNERZIP4","OWNERPHONE","OWNERPHONEEXT"]].fillna("Unknown",inplace=True)

/home/haslett/miniforge3/envs/rapids-24.10/lib/python3.12/site-packages/cudf/pandas/fast_slow_proxy.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  return fn(*args, **kwargs)
/home/haslett/miniforge3/envs/rapids-24.10/lib/python3.12/site-packages/cudf/pandas/fast_slow_proxy.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return fn(*args, **kwargs)


In [80]:
water_system_dict["Basic_Facility_Report_2023"][["LASTINSPECTION","LASTSANITARYSURVEY"]].value_counts().head(10)

LASTINSPECTION  LASTSANITARYSURVEY
2-Aug-23        8-Sep-22              7
20-May-92       20-May-92             6
1-Dec-23        24-Aug-21             6
28-Apr-22       28-Jul-23             4
24-Jan-89       24-Jan-89             4
4-Aug-23        8-Sep-22              4
21-Dec-23       26-Aug-21             4
8-May-17        2-Aug-23              4
19-May-92       19-May-92             3
29-Aug-14       6-Feb-24              3
Name: count, dtype: int64

### The last inspection dates seem to reside mostly in 2023 but there are still dates that frequently go back to 2012-2014 which is not recent at all, especially for an inspection. I would love to see more of 

### The same idea follow suit with the last inspection date for last survey date

## Plant Treatment Cleaning

In [81]:
water_system_dict["Plant_Treatment_Report_2023"].isnull().sum()

PWSID                      0
SYSTEMNAME                 0
PLANTNUMBER                0
PLANTNAME               2102
OBJECTIVEID                0
PROCESSID               1751
PROCESSOBJECTIVEDESC       0
dtype: int64

In [82]:
missing_column_display("Plant_Treatment_Report_2023")

,PLANTNAME,PROCESSID
0,PLANT #1,1.0
1,PLANT #1,1.0
2,PLANT #1,9.0
3,PLANT #1,10.0
4,PLANT #1,11.0
...,...,...
24945,CSU WTP 1,1.0
24946,CSU WTP 1,30.0
24947,CSU WTP 1,19.0
24948,CSU WTP 1,28.0


In [83]:
water_system_dict["Plant_Treatment_Report_2023"]["PLANTNAME"].value_counts().head(10)

PLANTNAME
PLANT #1         323
SAME             226
PLANT 1          225
WELL #1          123
WATER PLANT      105
LAKELAND          87
WTP               78
WINTER HAVEN      64
PLANT #2          64
SAME AS ABOVE     62
Name: count, dtype: int64

### If the name of the plant is null, then it can be labeled as unknown

In [84]:
water_system_dict["Plant_Treatment_Report_2023"]["PLANTNAME"].fillna("Unknown",inplace=True)

In [85]:
water_system_dict["Plant_Treatment_Report_2023"]["PROCESSID"].fillna(999,inplace=True)

### Keeping the Processid as a numeric data type but label the null values as 999 so that you can filter them easily during analysis

In [86]:
store_file("Plant_Treatment_Report_2023",1)

file has been saved
